##3.5 출력층 설계하기

**기계학습**
*   분류-데이터가 어느 클래스에 속하는지의 문제-항등함수 사용
*   회귀-입력 데이터에서 연속적인 수치를 예측함-소프트맥스 함수 사용



**항등 함수와 소프트맥스 함수 구현하기**

*   항등 함수: 입력을 그대로 출력
*   소프트백스 함수: 모든 입력신호로부터 화살표를 받음



In [2]:
#소프트맥스 함수
import numpy as np
a=np.array([0.3,2.9,4.0])
exp_a=np.exp(a)#지수 함수
print(exp_a)
sum_exp_a=np.sum(exp_a) # 지수 함수의 합
print(sum_exp_a)
y=exp_a/sum_exp_a
print(y)

[ 1.34985881 18.17414537 54.59815003]
74.1221542101633
[0.01821127 0.24519181 0.73659691]


In [ ]:
def softmax(a):
  exp_a=np.exp(a)
  sum_exp_a=np.sum(exp_a)
  y=exp_a/sum_exp_a
  return y

**소프트맥스 함수 수현시 주의점**

*   오버플로의 문제: 입력 신호 중 최댓값을 이용함




In [3]:
a=np.array([1010,1000,990])
print(np.exp(a)/np.sum(np.exp(a)))
c=np.max(a)
print(a-c)
print(np.exp(a-c)/np.sum(np.exp(a-c)))

[nan nan nan]
[  0 -10 -20]
[9.99954600e-01 4.53978686e-05 2.06106005e-09]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [4]:
#입력 신호 중 최댓값을 뺌
def softmax(a):
  c=np.max(a)
  exp_a=np.exp(a-c) 
  sum_exp_a=np.sum(exp_a)
  y=exp_a/sum_exp_a
  return y

**소프트맥스 함수의 특징**

In [5]:
a=np.array([0.3,2.9,4.0])
y=softmax(a)
print(y)
print(np.sum(y))

[0.01821127 0.24519181 0.73659691]
1.0




*   출력은 0에서 1.0 사이의 실수
*   출력 총합이 1이됨-->이 성질 덕분에 출력을 <확률>로 해석 가능



**출력층의 뉴런 수 정하기**

*   분류에서는 분류하고 싶은 클래스 수로 설정하는 것이 일반적




##3.6 손글씨 숫자 인식

*   추론과정만 구현 함(이 추론 과정을 신경망의 순전파라고도 함)




**MNIST 데이터 셋**

*   손글씨 숫자 이미지 집합




In [6]:
import sys, os
sys.path.append(os.pardir) #부모 디렉터리의 파일을 가져올 수 있도록 설정
from dataset.mnist import load_mnist

(x_train, t_train),(x_test, t_test)=\
  load_mnist(flatten+True, normalize=False)#읽은 데이터를 (훈련 이미지,레이블), (시험 이미지, 시험 레이블) 형식으로 반환
#normalize:입력 이미지의 픽셀값을 0.0~1.0사이로 정규화할지를 정함
#flatten:입력 이미지를 1차원 배열로 만들지를 정함
#one_hot_label:레이블을 원 핫 인코딩(정답을 뜻하는 원소만 1 나머지는 다 0) 형태로 저장할지를 정함
#각 데이터의 형상 출력
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

ModuleNotFoundError: ignored

In [7]:
import sys, os
sys.path.append(os.pardir)  
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))#저장된 이미지 데이터를 PIL용 데이터 객체로 변환
    pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

img = x_train[0]
label = t_train[0]
print(label)

print(img.shape) #(784,)
img = img.reshape(28, 28)#원래 이미지의 모양으로 변형-->이미지가 1차원 배열로 저장되어 있기 때문 
print(img.shape)#(28,28)
img_show(img)

ModuleNotFoundError: ignored

**신경망의 추론 처리**

In [8]:
def get_data():
    (x_train, t_train), (x_test, t_test)=\
      load_mnist(normalize=True, flatten=True, one_bot_label=False)
    return x_test, t_test

def init_network():#학습된 가중치 매개변수를 읽음
    with open("sample_weight.pkl", "rb") as f:
        network=pickle.load(f)
    
    return network

def predict(network, x):
    W1, W2, W3=network["W1"], network["W2"], network["W3"]
    b1, b2, b3=network["b1"], network["b2"], network["b3"]

    a1=np.dot(x, W1) + b1
    z1=sigmoid(a1)
    a2=np.dot(z1, W2) + b2
    z2=sigmoid(a2)
    a3=np.dot(z2, W3) + b3
    y=softmax(a3)

    return y

In [9]:
x, t=get_data()
network=init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y=predict(network,x[i])
    p=np.argmax(y)#확률이 가장 높은 원소의 인덱스를 받음
    if p==t[i]:
      accuracy_cnt +=1
print("Accuracy:"+str(float(accuracy_cnt)/len(x)))

NameError: ignored



*   정규화 : 데이터를 특정 범위로 변환하는 처리
*   전처리 : 신경망의 입력 데이터에 특정 변환을 가하는 것


**배치 처리**

In [10]:
x,t=get_data()
network=init_network()
W1,W2,W3=network['W1'],network['W2'],network['W3']
x.shape
x[0].shape
W1.shape
W2.shape
W3.shape
#q배치:하나로 묶은 입력 데이터

NameError: ignored

In [11]:
x, t = get_data()
network = init_network()

batch_size = 100#배치 크기
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:"+str(float(accuracy_cnt))/len(x))

NameError: ignored

In [12]:
list(range(0,10))
ist(range(0,10,3))

NameError: ignored

In [13]:
x=np.array([[0.1,0.8,0.1],[0.3,0.1,0.6],[0.2,0.5,0.3],[0.8,0.1,0.1]])
y=np.argmax(x,axis=1)
print(y)

[1 2 1 0]


In [14]:
y=np.array([1,2,1,0])
t=np.array([1,2,0,0])
print(y==t)
np.sum(y==t)

[ True  True False  True]


3

##3.7 정리

*  신경망에서는 활성화 함수로 시그모이드 함수나 ReLU함수 같은 매끄럽게 변화하는 함수를 이용한다
*   넘파이의 다차원 배열을 잘 사용하면 신경망을 효율적으로 구현 할 수 있다
*   기계학습 문제는 크게 회귀롸 분류로 나눌 수 있다
*   출력층의 활성화 함수로는 회귀에서는 주로 항등함수를, 분류에서는 주로 소프트맥스 함수를 이용한다
*   분류에서는 출력층의 뉴런 수를 분류하려는 클래스 수와 같게 설정한다
*   입력 데이터를 묶은 것을 배치라 하며, 추론 처리를 이 배치 단위로 진행하면 결과를 휠씬 빠르게 얻을 수 있다.







##4.1 데이터에서 학습한다

*   신경망의 특징:데이터를 보고 학습할 수 있음
*   =데이터 가중치 매개변수의 값을 데이터를 보고 자동으로 결정함



**데이터 주도 학습**

*   기계학습:모아진 데이터로 부터 규칙을 찾아내는 역할을 기계가 담당함(이미지를 벡터로 변환할 때 사용하는 특징은 사람이 설계함)
*   신경망:이미지의 특징까지 기계가 학습함



**훈련 데이터와 시험 데이터**

*   훈련 데이터로 학습하고 시험데이터로 범용 능력을 평가함
*   범용능력:아직 보지못한 데이터로도 문제를 올바르게 풀어내는 능력
*   오버피팅:한 데이터셋에만 지나치게 최적화된 상태




##4.2 손실 함수
최적의 매개변수 값을 탐색할때 기준으로 삼는 지표

**오차제곱합**

각 원소의 출력(추정 값)과 정답 레이블(참 값)의 차를 제곱한 후 그 총함을 구함

In [ ]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [15]:
def sum_squares_error(y, t):
  return 0.5*np.sum((y-t)**2)

In [18]:
#정답은 2
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
#예1: 2일 확률이 가장 높다고 추정함
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

0.09750000000000003

In [19]:
#예2: 7일 확률이 가장 높다고 추정함
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

0.5975

**교차 엔트로피 오차**


*   정답일 때의 추정의 자연로그를 계산하는 식
*   정답에 해당하는 출력이 커질수록 0에 다가가다가 그 출력이 1일때 0이 됨/정답일 때 출력이 작아질수록 오차가 커짐


In [20]:
def cross_entropy_error(y, t):
  delta = 1e-7
  return -np.sum(t*np.log(y+delta))

In [21]:
#정답일 때 출력이 0.6
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [22]:
#정답일 때 출력이 0.1
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

**미니베치 학습**

*   훈련데이터를 통해 학습하려면 모든 훈련데의터의 손실 함수 값을 구해야함
*   일일이 손실함수를 계산하는 건 불가능 함으로 훈련데이터로부터 일부만 골라 학습을 수행-이러한 방법을 미니배치 학습, 골라진 데이터를 미니배치라고 함



In [23]:
import sys, os
sys.path.append(os.pardir)  
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
  load_mnist(normalize=True, one_hot_abel=True)

print(x_train.shape) #(60000, 784)
print(t_train.shape) #(60000, 10)

ModuleNotFoundError: ignored

In [24]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

NameError: ignored

**(배치용) 교차 엔트로피 구현하기**

In [25]:
#정답레이블이 원 핫 인코딩
def cross_entropy_error2(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)

  batxh_size = y.shape[0]
  return -np.sum(t*np.log(y+1e-7))/batch_size

In [ ]:
#정답레이블이 숫자 레이블
def cross_entropy_error2(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)

  batxh_size = y.shape[0]
  return -np.sum(t*np.log(y[np.arrange(batch_size), t]+1e-7))/batch_size

**왜 손실 함수를 설정하는가**


*   신경망 학습에서는 최적의 매개변수를 탐색할 때 매개변수의 미분을 계산하고 그 미분값을 단서로 매개변수의 값을 서서히 갱신함
*   미분 값이 대부분의 장소에서 0이되어 매개변수를 갱신할 수 없으므로 정확도를 지표로 삼을 수 없음
*   즉 정확도는 매개변수의 미소한 변화에는 반응을 보이지 않고 반응이 있더라도 그 값이 불연속적으로 갑자기 변화함
*   계단함수를 활성화 함수로 사용하지 않는 이유와도 동일


